In [ ]:
import arcgis
import pandas as pd
import numpy as np

In [ ]:
with open(r"..\data.json", "r") as f:
    df = pd.read_json(f, orient="records", date_unit="ms")
df = arcgis.features.SpatialDataFrame(df)

In [ ]:
def dissolve_sdf(df, fields):
    def dissolve_shapes(row, sr):
        tmp = arcgis.gis.GIS()
        geom = arcgis.geometry.functions.union(geometries=row.SHAPE, spatial_ref=sr, gis=tmp)
        return geom
    
    na = df.fillna("NONE")

    grouped = na.groupby(fields)

    listed_geoms = grouped.SHAPE.apply(list)
    listed_geoms = lisetd_geoms.reset_index()

    dissolved = listed_geoms.replace("NONE", np.NaN)

    dissolved.SHAPE = dissolved.apply(dissolve_shapes, sr=2876, axis=1)

    return dissolved

In [ ]:
diss = dissolve_sdf(df, ["FLOODPLAIN", "FEMAZONE", "DRAINAGE", "LIFECYCLE", "ADOPTDATE", "INEFFDATE", "SOURCE"])
diss